# 问题一

In [1]:
import pandas as pd
import numpy as np
from sklearn import feature_selection
from scipy import stats
import matplotlib.pyplot as plt


plt.rcParams['font.sans-serif'] = ['SimHei']  # 中文字体设置-黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决保存图像是负号'-'显示为方块的问题
# sns.set(font='SimHei', font_scale=0.8)        # 解决Seaborn中文显示问题

## 问1：对这些玻璃文物的表面风化与其玻璃类型、纹饰和颜色的关系进行分析

### 进行卡方检验

In [2]:
# 读取数据
data111 = pd.read_excel('./data/style_name.xlsx')
data111.head()

,文物编号,纹饰,类型,颜色,表面风化
0,1,C,高钾,蓝绿,无风化
1,2,A,铅钡,浅蓝,风化
2,3,A,高钾,蓝绿,无风化
3,4,A,高钾,蓝绿,无风化
4,5,A,高钾,蓝绿,无风化


In [3]:
print(data111.isna().any(axis=0))
data111.fillna('未知', inplace=True)
data111.isna().any(axis=0)

文物编号    False
纹饰      False
类型      False
颜色       True
表面风化    False
dtype: bool


文物编号    False
纹饰      False
类型      False
颜色      False
表面风化    False
dtype: bool

In [4]:
# 进行数字符号化
# yanse  = {'蓝绿':3, '浅蓝':1, '黑':7, '深绿':4, '浅绿':0, '紫':6, '深蓝':5, '绿':2, '未知颜色':8}
# biaomianfenghua = {'无风化':0, '风化':1}
# leixing = {'高钾':0, '铅钡':1}
# wen_lei_yan_feng = {"A":0, "B":1, "C":2, '高钾':0, '铅钡':1, '蓝绿':3, '浅蓝':1, '黑':7, '深绿':4, '浅绿':0, '紫':6, '深蓝':5, '绿':2, '未知':8, '无风化':0, '风化':1}
# data111.replace(wen_lei_yan_feng, inplace=True)
# data111.head()

In [5]:
# 求列联表
tab_wenshi = pd.crosstab(data111.iloc[:, 1], data111.iloc[:, -1])
tab_leixing = pd.crosstab(data111.iloc[:, 2], data111.iloc[:, -1])
tab_yanse = pd.crosstab(data111.iloc[:, 3], data111.iloc[:, -1])
tabs111 = [tab_wenshi, tab_leixing, tab_yanse]
[print(tab, '\n') for tab in tabs111]

表面风化  无风化  风化
纹饰           
A      11  11
B       0   6
C      13  17 

表面风化  无风化  风化
类型           
铅钡     12  28
高钾     12   6 

表面风化  无风化  风化
颜色           
未知      0   4
浅绿      2   1
浅蓝      8  12
深绿      3   4
深蓝      2   0
紫       2   2
绿       1   0
蓝绿      6   9
黑       0   2 



[None, None, None]

In [6]:
# 得到 类型 与 是否风化 相关 其次是 纹饰
shuxing_chi2 = [stats.chi2_contingency(tab.values)[0:3] for tab in tabs111]
print('           F                   p-value              自由度')
[*zip(['纹饰', '类型', '颜色'], shuxing_chi2)]

           F                   p-value              自由度


[('纹饰', (4.9565359477124185, 0.08388839673210007, 2)),
 ('类型', (5.451814406318083, 0.019548014331003307, 1)),
 ('颜色', (9.432446311858078, 0.3071378142880555, 8))]

### 绘制分类图

In [7]:
# 绘图
def pplot(tab):
    index = tab.index
    data = tab.values
    columns = tab.columns
    fig = plt.figure(dpi=400)
    ax = plt.axes()
    ax.barh(index, data[:, 0])
    ax.barh(index, -data[:, 1])
    ax.set_xticklabels([str(abs(int(x))) for x in ax.get_xticks()])
    ax.legend(columns)
    ax.grid()
    plt.xlabel('数量')
    plt.ylabel(index.name)
    plt.savefig('./imgs/问题一/'+index.name+'.jpg', dpi=400, bbox_inches='tight')
    plt.close()

In [8]:
[pplot(tab) for tab in tabs111]

C:\Users\airri\AppData\Local\Temp\ipykernel_6932\653229712.py:10: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([str(abs(int(x))) for x in ax.get_xticks()])
C:\Users\airri\AppData\Local\Temp\ipykernel_6932\653229712.py:10: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([str(abs(int(x))) for x in ax.get_xticks()])
C:\Users\airri\AppData\Local\Temp\ipykernel_6932\653229712.py:10: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([str(abs(int(x))) for x in ax.get_xticks()])


[None, None, None]

In [9]:
# # 选择 k 个最佳特征
# fea_sele = feature_selection.SelectKBest(feature_selection.chi2, k=1)
# shuxing111 = data111.iloc[:, 1:-1].values
# target111 = data111.iloc[:, -1].values
# # shuxing111 是特征数据，target111 是标签数据，该函数可以选择出 k 个最佳特征
# fea_sele_output = fea_sele.fit_transform(shuxing111, target111)
# # 得分
# print("Score: {},\n P: {}".format(fea_sele.scores_, fea_sele.pvalues_))

## 问2：结合玻璃的类型，分析文物样品表面有无风化化学成分含量的统计规律

### 高钾

In [10]:
data121 = pd.read_excel('./data/gaojia_elem.xlsx')
data121.head()

,文物编号,纹饰,类型,颜色,表面风化,文物采样点,采样类别,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),...,氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2),二氧化硫(SO2),成分总和
0,1,C,高钾,蓝绿,无风化,01,无风化,69.33,0.0,9.99,...,3.93,1.74,3.87,0.00,0.00,1.17,0.00,0.0,0.39,97.61
1,3,A,高钾,蓝绿,无风化,03部位1,无风化,87.05,0.0,5.19,...,4.06,0.00,0.78,0.25,0.00,0.66,0.00,0.0,0.00,100.00
2,3,A,高钾,蓝绿,无风化,03部位2,无风化,61.71,0.0,12.37,...,5.50,2.16,5.09,1.41,2.86,0.70,0.10,0.0,0.00,98.88
3,4,A,高钾,蓝绿,无风化,04,无风化,65.88,0.0,9.67,...,6.44,2.06,2.18,0.00,0.00,0.79,0.00,0.0,0.36,96.06
4,5,A,高钾,蓝绿,无风化,05,无风化,61.58,0.0,10.95,...,7.50,2.62,3.27,0.00,0.00,0.94,0.06,0.0,0.47,96.51


In [11]:
fenghua121 = data121['采样类别']
fenghua121_index = fenghua121 == '风化'
feng_elem121 = data121.loc[fenghua121_index, :].iloc[:, 7:-1]
feng_elem121_analysis = pd.DataFrame([feng_elem121.mean(axis=0), feng_elem121.median(), feng_elem121.std(ddof=1)], index=['平均值', '中位数', '标准差'])
feng_elem121_analysis

,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),氧化钙(CaO),氧化镁(MgO),氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2),二氧化硫(SO2)
平均值,93.963333,0.0,0.543333,0.87000,0.196667,1.93000,0.265000,1.561667,0.0,0.0,0.280000,0.0,0.0,0.0
中位数,93.505000,0.0,0.665000,0.83000,0.000000,1.72000,0.275000,1.545000,0.0,0.0,0.280000,0.0,0.0,0.0
标准差,1.733617,0.0,0.445182,0.48777,0.306311,0.96449,0.069498,0.934824,0.0,0.0,0.209952,0.0,0.0,0.0


In [12]:
wufeng121_index = fenghua121 == '无风化'
wufeng_elem121 = data121.loc[wufeng121_index, :].iloc[:, 7:-1]
wufeng_elem121_analysis = pd.DataFrame([wufeng_elem121.mean(axis=0), wufeng_elem121.median(), wufeng_elem121.std(ddof=1)], index=['平均值', '中位数', '标准差'])
wufeng_elem121_analysis

,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),氧化钙(CaO),氧化镁(MgO),氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2),二氧化硫(SO2)
平均值,67.984167,0.695000,9.330833,5.332500,1.079167,6.620000,1.931667,2.45250,0.411667,0.598333,1.402500,0.041667,0.196667,0.101667
中位数,65.530000,0.000000,9.830000,6.095000,1.165000,6.185000,2.110000,2.34500,0.155000,0.000000,1.020000,0.020000,0.000000,0.000000
标准差,8.755099,1.286917,3.920303,3.092478,0.676145,2.491509,1.666694,1.65999,0.588987,0.982102,1.433959,0.048399,0.681273,0.185513


### 铅钡

In [13]:
data122 = pd.read_excel('./data/qianbei_elem.xlsx')
data122.head()

,文物编号,纹饰,类型,颜色,表面风化,文物采样点,采样类别,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),...,氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2),二氧化硫(SO2),成分总和
0,2,A,铅钡,浅蓝,风化,02,风化,36.28,0.0,1.05,...,5.73,1.86,0.26,47.43,0.00,3.57,0.19,0.0,0.00,99.89
1,8,C,铅钡,紫,风化,08,风化,20.14,0.0,0.00,...,1.34,0.00,10.41,28.68,31.23,3.59,0.37,0.0,2.58,99.82
2,8,C,铅钡,紫,风化,08严重风化点,严重风化,4.61,0.0,0.00,...,1.11,0.00,3.14,32.45,30.62,7.56,0.53,0.0,15.03,98.24
3,11,C,铅钡,浅蓝,风化,11,风化,33.59,0.0,0.21,...,2.69,0.00,4.93,25.39,14.61,9.38,0.37,0.0,0.00,95.39
4,19,A,铅钡,NaN,风化,19,风化,29.64,0.0,0.00,...,3.57,1.33,3.51,42.82,5.35,8.83,0.19,0.0,0.00,98.76


In [14]:
fenghua122 = data122['采样类别']
fenghua122_index = fenghua122 == '风化'
feng_elem122 = data122.loc[fenghua122_index, :].iloc[:, 7:-1]
feng_elem122_analysis = pd.DataFrame([feng_elem122.mean(axis=0), feng_elem122.median(), feng_elem122.std(ddof=1)], index=['平均值', '中位数', '标准差'])
feng_elem122_analysis

,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),氧化钙(CaO),氧化镁(MgO),氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2),二氧化硫(SO2)
平均值,27.056087,0.244348,0.133478,2.777391,0.686522,3.099130,0.660870,2.221304,43.710000,10.474783,4.760000,0.374348,0.077391,0.197391
中位数,25.740000,0.000000,0.000000,2.820000,0.590000,2.510000,0.330000,0.880000,44.120000,8.640000,3.590000,0.390000,0.000000,0.000000
标准差,9.005117,0.587288,0.246145,1.666628,0.719499,2.746689,0.750847,2.980539,12.077451,7.965839,3.988562,0.229977,0.285979,0.660642


In [15]:
wufeng122_index = fenghua122 == '无风化'
wufeng_elem122 = data122.loc[wufeng122_index, :].iloc[:, 7:-1]
wufeng_elem122_analysis = pd.DataFrame([wufeng_elem122.mean(axis=0), wufeng_elem122.median(), wufeng_elem122.std(ddof=1)], index=['平均值', '中位数', '标准差'])
wufeng_elem122_analysis

,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),氧化钙(CaO),氧化镁(MgO),氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2),二氧化硫(SO2)
平均值,54.659565,1.682609,0.218696,1.320435,0.640435,4.456087,0.736522,1.431739,22.084783,9.001739,1.049130,0.268261,0.046522,0.159130
中位数,54.610000,0.000000,0.150000,0.840000,0.710000,3.860000,0.000000,0.650000,20.120000,8.990000,0.190000,0.260000,0.000000,0.000000
标准差,11.828590,2.371636,0.310063,1.284698,0.546755,3.262449,1.154732,1.969868,8.215147,5.825285,1.847056,0.243453,0.127337,0.763163


In [16]:
yanfeng122_index = fenghua122 == '严重风化'
yanfeng_elem122 = data122.loc[yanfeng122_index, :].iloc[:, 7:-1]
yanfeng_elem122_analysis = pd.DataFrame([yanfeng_elem122.mean(axis=0), yanfeng_elem122.median(), yanfeng_elem122.std(ddof=1)], index=['平均值', '中位数', '标准差'])
yanfeng_elem122_analysis

,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),氧化钙(CaO),氧化镁(MgO),氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2),二氧化硫(SO2)
平均值,8.480000,0.0,0.133333,2.066667,0.370000,1.980000,0.0,2.693333,40.276667,22.023333,9.243333,0.756667,0.0,10.326667
中位数,4.610000,0.0,0.000000,3.010000,0.000000,1.180000,0.0,3.140000,32.450000,30.620000,7.560000,0.620000,0.0,15.030000
标准差,7.487035,0.0,0.230940,1.792047,0.640859,1.446686,0.0,1.194376,15.797957,19.225052,4.299678,0.317857,0.0,8.954978


## 问3：根据风化点检测数据，预测其风化前的化学成分含量。

In [17]:
def predict_elem(df, type, asend):
    df = df.sort_values(by='二氧化硅(SiO2)', ascending=asend)
    # print(df)
    X = np.arange(1, len(df)+1, 1)
    num = len(df.columns)
    Y = []
    P = []
    for i in range(1, num):
        y = df.iloc[:, i].values
        f1 = np.polyfit(X, y, 6)
        p1 = np.poly1d(f1)
        xx = len(df.loc[df['采样类别']=='无风化']) + 0.5
        yy = p1(xx)
        figure = plt.figure(dpi=400)
        _ = plt.plot(X, p1(X), 'b-')
        _ = plt.plot(X, y, 'mo')
        _ = plt.plot(xx, yy, 'r*', markersize='10')
        plt.legend(['预测曲线', '真实数据', '预测数据'])
        plt.grid()
        plt.xlabel('风化程度')
        plt.ylabel('成分含量%')
        ax = plt.gca()
        ax.set_xticklabels([' ' for x in ax.get_xticks()])
        plt.savefig('./imgs/问题二/'+ type + '/' + df.columns[i]+'随风化程度拟合.jpg', dpi=400, bbox_inches='tight')
        plt.close()
        P.append(p1)
        Y.append(yy)
    Y = np.clip(Y, 0, +np.inf)
    return P, Y

### 高钾

In [18]:
data131 = pd.read_excel('./data/gaojia_elem.xlsx')
data131.head()

,文物编号,纹饰,类型,颜色,表面风化,文物采样点,采样类别,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),...,氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2),二氧化硫(SO2),成分总和
0,1,C,高钾,蓝绿,无风化,01,无风化,69.33,0.0,9.99,...,3.93,1.74,3.87,0.00,0.00,1.17,0.00,0.0,0.39,97.61
1,3,A,高钾,蓝绿,无风化,03部位1,无风化,87.05,0.0,5.19,...,4.06,0.00,0.78,0.25,0.00,0.66,0.00,0.0,0.00,100.00
2,3,A,高钾,蓝绿,无风化,03部位2,无风化,61.71,0.0,12.37,...,5.50,2.16,5.09,1.41,2.86,0.70,0.10,0.0,0.00,98.88
3,4,A,高钾,蓝绿,无风化,04,无风化,65.88,0.0,9.67,...,6.44,2.06,2.18,0.00,0.00,0.79,0.00,0.0,0.36,96.06
4,5,A,高钾,蓝绿,无风化,05,无风化,61.58,0.0,10.95,...,7.50,2.62,3.27,0.00,0.00,0.94,0.06,0.0,0.47,96.51


In [19]:
df131 = data131.iloc[:, 6:-1]
P, Y = predict_elem(df131, '高钾', asend=True)
import sympy as sy
x = sy.symbols('x')
print(df131.columns)
print([sy.expand(P[i](x)) for i in range(len(P))], '\n', Y)

C:\Users\airri\AppData\Local\Temp\ipykernel_6932\1445848202.py:23: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([' ' for x in ax.get_xticks()])


Index(['采样类别', '二氧化硅(SiO2)', '氧化钠(Na2O)', '氧化钾(K2O)', '氧化钙(CaO)', '氧化镁(MgO)',
       '氧化铝(Al2O3)', '氧化铁(Fe2O3)', '氧化铜(CuO)', '氧化铅(PbO)', '氧化钡(BaO)',
       '五氧化二磷(P2O5)', '氧化锶(SrO)', '氧化锡(SnO2)', '二氧化硫(SO2)'],
      dtype='object')
[0.000101176476820061*x**6 - 0.00522841557484881*x**5 + 0.0985884186658272*x**4 - 0.833803714515793*x**3 + 3.30682982200545*x**2 - 4.6554174930049*x + 61.1230555555518, 5.93862640403153e-5*x**6 - 0.0035701163320455*x**5 + 0.0829756430677549*x**4 - 0.932559980205655*x**3 + 5.1633769528164*x**2 - 12.7525440529294*x + 11.1902865761667, 7.80041803143765e-5*x**6 - 0.00469693375061652*x**5 + 0.109704456820857*x**4 - 1.23252819749168*x**3 + 6.62128699652928*x**2 - 15.0764267734525*x + 21.4002941176441, 5.34448333108968e-5*x**6 - 0.0032828200986431*x**5 + 0.0777419032654395*x**4 - 0.882113585532741*x**3 + 4.82026723408921*x**2 - 11.5890257307627*x + 16.0181749622911, -3.29232474835818e-5*x**6 + 0.00194541520841725*x**5 - 0.0443101195951252*x**4 + 0.48786072738586*x*

### 铅钡

In [20]:
data132 = pd.read_excel('./data/qianbei_elem.xlsx')
data132.head()
df132 = data132.iloc[:, 6:-1]
P, Y = predict_elem(df132, '铅钡', asend=False)
import sympy as sy
x = sy.symbols('x')
print(df132.columns)
print([sy.expand(P[i](x)) for i in range(len(P))], '\n', Y)

C:\Users\airri\AppData\Local\Temp\ipykernel_6932\1445848202.py:23: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([' ' for x in ax.get_xticks()])


Index(['采样类别', '二氧化硅(SiO2)', '氧化钠(Na2O)', '氧化钾(K2O)', '氧化钙(CaO)', '氧化镁(MgO)',
       '氧化铝(Al2O3)', '氧化铁(Fe2O3)', '氧化铜(CuO)', '氧化铅(PbO)', '氧化钡(BaO)',
       '五氧化二磷(P2O5)', '氧化锶(SrO)', '氧化锡(SnO2)', '二氧化硫(SO2)'],
      dtype='object')
[3.85766026860884e-8*x**6 - 9.17118754606129e-6*x**5 + 0.000712616857594223*x**4 - 0.0240010689158373*x**3 + 0.368128216206803*x**2 - 3.78024518582681*x + 77.418712278836, 9.47732090986127e-8*x**6 - 1.50598185911019e-5*x**5 + 0.000905075831001527*x**4 - 0.0251066434078937*x**3 + 0.306198689141364*x**2 - 1.17820051722263*x + 1.24982193182645, -7.47429653437481e-9*x**6 + 1.11433363006402e-6*x**5 - 6.07485357232117e-5*x**4 + 0.00146733031125077*x**3 - 0.0152866571939529*x**2 + 0.0620147856121915*x + 0.0881622774532421, -2.10410059865739e-9*x**6 + 8.33145236602823e-7*x**5 - 8.64254796313041e-5*x**4 + 0.00367442525143033*x**3 - 0.0676733926893626*x**2 + 0.518539913724026*x - 0.0163594063577871, 7.21564120925123e-9*x**6 - 1.32095614609852e-6*x**5 + 8.4098573764725

# 问题二

In [21]:
import pandas as pd
import numpy as np
from scipy import stats, spatial
import matplotlib.pyplot as plt
from sklearn import preprocessing, cluster, model_selection, ensemble

plt.rcParams['font.sans-serif'] = ['SimHei']  # 中文字体设置-黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决保存图像是负号'-'显示为方块的问题
# sns.set(font='SimHei', font_scale=0.8)        # 解决Seaborn中文显示问题

## 问1：依据附件数据分析高钾玻璃、铅钡玻璃的分类规律

### 对风化数据进行分类 高钾-铅钡分类

In [22]:
# 风化数据
data211 = pd.read_excel("./data/fenghua.xlsx")
data211.head()

,文物编号,纹饰,类型,颜色,表面风化,文物采样点,采样类别,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),...,氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2),二氧化硫(SO2),成分总和
0,2,A,铅钡,浅蓝,风化,02,风化,36.28,0.0,1.05,...,5.73,1.86,0.26,47.43,0.00,3.57,0.19,0.0,0.00,99.89
1,7,B,高钾,蓝绿,风化,07,风化,92.63,0.0,0.00,...,1.98,0.17,3.24,0.00,0.00,0.61,0.00,0.0,0.00,99.70
2,8,C,铅钡,紫,风化,08,风化,20.14,0.0,0.00,...,1.34,0.00,10.41,28.68,31.23,3.59,0.37,0.0,2.58,99.82
3,8,C,铅钡,紫,风化,08严重风化点,严重风化,4.61,0.0,0.00,...,1.11,0.00,3.14,32.45,30.62,7.56,0.53,0.0,15.03,98.24
4,9,B,高钾,蓝绿,风化,09,风化,95.02,0.0,0.59,...,1.32,0.32,1.55,0.00,0.00,0.35,0.00,0.0,0.00,99.77


In [23]:
# 高钾 铅钡 共两类
x211 = data211.iloc[:, 7:-1]
klf211 = cluster.KMeans(n_clusters=2, random_state=10)
klf211.fit(x211)
y_pred211 = klf211.predict(x211).astype(dtype=bool)
lei211_1 = x211.loc[y_pred211==True, :]
lei211_2 = x211.loc[y_pred211==False, :]
result211 = [*zip(x211.columns, [stats.levene(lei211_1.iloc[:, i], lei211_2.iloc[:, i])[0:2] for i in range(x211.shape[1])])]

result211

[('二氧化硅(SiO2)', (0.0058785771263199504, 0.9393936352722135)),
 ('氧化钠(Na2O)', (0.12736795824819983, 0.7236749598740341)),
 ('氧化钾(K2O)', (4.266054174127505, 0.047614113940546474)),
 ('氧化钙(CaO)', (4.36448591183703, 0.04527898592786178)),
 ('氧化镁(MgO)', (0.7459983831537677, 0.3945993595929932)),
 ('氧化铝(Al2O3)', (1.5592851928268912, 0.22142681898329655)),
 ('氧化铁(Fe2O3)', (2.5117202770542457, 0.12348867407181417)),
 ('氧化铜(CuO)', (1.0006330048596477, 0.3251580121499186)),
 ('氧化铅(PbO)', (4.181336269521442, 0.04973140606702856)),
 ('氧化钡(BaO)', (3.5947729490690836, 0.06763026318527048)),
 ('五氧化二磷(P2O5)', (16.22109087295957, 0.0003539901886993054)),
 ('氧化锶(SrO)', (5.598601826091504, 0.02463079786661358)),
 ('氧化锡(SnO2)', (2.7625670483644567, 0.10690900420515216)),
 ('二氧化硫(SO2)', (0.7521291582221032, 0.39268715571916846))]

In [24]:
lei211_1 = x211.loc[data211['类型'] == '高钾', :]
lei211_2 = x211.loc[data211['类型'] == '铅钡', :]
result211 = [*zip(x211.columns, [stats.levene(lei211_1.iloc[:, i], lei211_2.iloc[:, i])[0:2] for i in range(x211.shape[1])])]
result211

[('二氧化硅(SiO2)', (5.257285923543327, 0.029039399020675392)),
 ('氧化钠(Na2O)', (0.8821065127890421, 0.3551263298219277)),
 ('氧化钾(K2O)', (3.696884098835635, 0.06405350811360071)),
 ('氧化钙(CaO)', (5.927338233490042, 0.02106991697287744)),
 ('氧化镁(MgO)', (4.449839059070152, 0.04335677602262209)),
 ('氧化铝(Al2O3)', (0.9335910919177165, 0.3416566151966325)),
 ('氧化铁(Fe2O3)', (3.9765554859274164, 0.05529604331238446)),
 ('氧化铜(CuO)', (1.1097387686824056, 0.3005498299126895)),
 ('氧化铅(PbO)', (7.4612244187795085, 0.010457756663528463)),
 ('氧化钡(BaO)', (4.184105050708112, 0.049660574375098764)),
 ('五氧化二磷(P2O5)', (15.826562359026232, 0.000405340320971414)),
 ('氧化锶(SrO)', (8.033225111105196, 0.008139151861044714)),
 ('氧化锡(SnO2)', (0.37764124796582593, 0.5435013259157311)),
 ('二氧化硫(SO2)', (0.617166149266825, 0.43826326062970333))]

In [25]:
# 随机森林
df211 = data211.iloc[:, 7:-1]
type211 = data211['类型'].replace({'高钾':0, '铅钡':1})
x_train211, x_test211, y_train211, y_test211 = model_selection.train_test_split(df211, type211, test_size=0.3)
clf211 = ensemble.RandomForestClassifier(n_estimators=50, criterion='gini', random_state=10)
clf211 = clf211.fit(x_train211, y_train211)
clf211.score(x_test211, y_test211)

1.0

In [26]:
feature_name211 = df211.columns
result211 = [*zip(feature_name211, (clf211.feature_importances_ * 100).astype(np.int32))]
result211
# fig = plt.figure(dpi=400)
# plt.barh(feature_name211, (clf211.feature_importances_ * 100).astype(np.int32))
# plt.grid()

[('二氧化硅(SiO2)', 29),
 ('氧化钠(Na2O)', 0),
 ('氧化钾(K2O)', 2),
 ('氧化钙(CaO)', 8),
 ('氧化镁(MgO)', 0),
 ('氧化铝(Al2O3)', 2),
 ('氧化铁(Fe2O3)', 2),
 ('氧化铜(CuO)', 0),
 ('氧化铅(PbO)', 23),
 ('氧化钡(BaO)', 11),
 ('五氧化二磷(P2O5)', 3),
 ('氧化锶(SrO)', 14),
 ('氧化锡(SnO2)', 0),
 ('二氧化硫(SO2)', 0)]

### 对无风化数据进行分类 高钾-铅钡分类

In [27]:
# 无风化数据
data212 = pd.read_excel("./data/wufenghua.xlsx")
data212.head()

,文物编号,纹饰,类型,颜色,表面风化,文物采样点,采样类别,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),...,氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2),二氧化硫(SO2),成分总和
0,1,C,高钾,蓝绿,无风化,01,无风化,69.33,0.0,9.99,...,3.93,1.74,3.87,0.00,0.00,1.17,0.00,0.0,0.39,97.61
1,3,A,高钾,蓝绿,无风化,03部位1,无风化,87.05,0.0,5.19,...,4.06,0.00,0.78,0.25,0.00,0.66,0.00,0.0,0.00,100.00
2,3,A,高钾,蓝绿,无风化,03部位2,无风化,61.71,0.0,12.37,...,5.50,2.16,5.09,1.41,2.86,0.70,0.10,0.0,0.00,98.88
3,4,A,高钾,蓝绿,无风化,04,无风化,65.88,0.0,9.67,...,6.44,2.06,2.18,0.00,0.00,0.79,0.00,0.0,0.36,96.06
4,5,A,高钾,蓝绿,无风化,05,无风化,61.58,0.0,10.95,...,7.50,2.62,3.27,0.00,0.00,0.94,0.06,0.0,0.47,96.51


In [28]:
# 高钾 铅钡 共两类
x212 = data212.iloc[:, 7:-1]
klf212 = cluster.KMeans(n_clusters=2, random_state=10)
klf212.fit(x212)
y_pred212 = klf212.predict(x212).astype(dtype=bool)
lei212_1 = x212.loc[y_pred212==True, :]
lei212_2 = x212.loc[y_pred212==False, :]
result212 = [*zip(x212.columns, [stats.levene(lei212_1.iloc[:, i], lei212_2.iloc[:, i])[0:2] for i in range(x212.shape[1])])]
result212

[('二氧化硅(SiO2)', (0.41889738577180935, 0.5219648902216465)),
 ('氧化钠(Na2O)', (4.039203354130564, 0.05268909648961041)),
 ('氧化钾(K2O)', (30.011676262236122, 4.4950401288819706e-06)),
 ('氧化钙(CaO)', (5.739767695962669, 0.02241400782313376)),
 ('氧化镁(MgO)', (0.6276858898436387, 0.4338641559714449)),
 ('氧化铝(Al2O3)', (3.558429063840653, 0.06806960201924186)),
 ('氧化铁(Fe2O3)', (1.4922689662509068, 0.23051998616951747)),
 ('氧化铜(CuO)', (0.24949135614734802, 0.6207475215785672)),
 ('氧化铅(PbO)', (0.14058799401925828, 0.7100962122412668)),
 ('氧化钡(BaO)', (0.8057138103441971, 0.3758929160416895)),
 ('五氧化二磷(P2O5)', (1.298042587436155, 0.26277392151133344)),
 ('氧化锶(SrO)', (5.719974310754427, 0.02262929808412777)),
 ('氧化锡(SnO2)', (0.1354247330472715, 0.7152225176400975)),
 ('二氧化硫(SO2)', (1.0352922292110645, 0.31632105723960446))]

In [29]:
lei212_1 = x212.loc[data212['类型'] == '高钾', :]
lei212_2 = x212.loc[data212['类型'] == '铅钡', :]
result212 = [*zip(x212.columns, [stats.levene(lei212_1.iloc[:, i], lei212_2.iloc[:, i])[0:2] for i in range(x212.shape[1])])]
result212

[('二氧化硅(SiO2)', (1.6384037048245694, 0.20946996579993904)),
 ('氧化钠(Na2O)', (1.7879623040551529, 0.1903193147208077)),
 ('氧化钾(K2O)', (21.631759868257042, 5.139371070806594e-05)),
 ('氧化钙(CaO)', (7.768701661959397, 0.008748786266244531)),
 ('氧化镁(MgO)', (0.6264972756642464, 0.43429568967530274)),
 ('氧化铝(Al2O3)', (0.15043525624815915, 0.7006104422259531)),
 ('氧化铁(Fe2O3)', (0.9748429366693179, 0.3306597187703154)),
 ('氧化铜(CuO)', (0.18114273018882465, 0.6731559931159627)),
 ('氧化铅(PbO)', (16.598642801194885, 0.00027246597003490575)),
 ('氧化钡(BaO)', (7.18672322404747, 0.011374692702751838)),
 ('五氧化二磷(P2O5)', (0.04863673115693299, 0.8268114384397806)),
 ('氧化锶(SrO)', (6.1246110749575235, 0.01864194204734005)),
 ('氧化锡(SnO2)', (1.0740132292891047, 0.30757549350422775)),
 ('二氧化硫(SO2)', (0.06513895558879434, 0.8001346148400783))]

In [30]:
# 随机森林
df212 = data212.iloc[:, 7:-1]
type212 = data212['类型'].replace({'高钾':0, '铅钡':1})
x_train212, x_test212, y_train212, y_test212 = model_selection.train_test_split(df212, type212, test_size=0.3)
clf212 = ensemble.RandomForestClassifier(n_estimators=50, criterion='gini', random_state=10)
clf212 = clf212.fit(x_train212, y_train212)
clf212.score(x_test212, y_test212)

1.0

In [31]:
feature_name212 = df212.columns
result212 = [*zip(feature_name212, (clf212.feature_importances_ * 100).astype(np.int32))]
result212

[('二氧化硅(SiO2)', 2),
 ('氧化钠(Na2O)', 0),
 ('氧化钾(K2O)', 21),
 ('氧化钙(CaO)', 26),
 ('氧化镁(MgO)', 1),
 ('氧化铝(Al2O3)', 3),
 ('氧化铁(Fe2O3)', 1),
 ('氧化铜(CuO)', 0),
 ('氧化铅(PbO)', 15),
 ('氧化钡(BaO)', 20),
 ('五氧化二磷(P2O5)', 0),
 ('氧化锶(SrO)', 7),
 ('氧化锡(SnO2)', 0),
 ('二氧化硫(SO2)', 0)]

## 问2：对于每个类别选择合适的化学成分对其进行亚类划分，给出具体的划分方法及划分结果

### 集成函数

In [32]:
# 肘部法则
def elbow_analy(x, name):
    t = []
    for k in range(1, 6):
        k_mean_alg = cluster.KMeans(n_clusters=k, random_state=10)
        k_mean_alg.fit(x)
        # y_pre = k_mean_alg.predict(x)
        t.append(sum(np.min(spatial.distance.cdist(x, k_mean_alg.cluster_centers_, 'euclidean'), axis=1)) / x.shape[0])
    fig = plt.figure(dpi=400)
    _ = plt.plot(np.arange(1, len(t)+1) ,t, 'ro-')
    plt.xlabel('聚类 K 值')
    plt.ylabel('误差平方和')
    plt.grid()
    plt.savefig("./imgs/问题二/肘部法则-" + name + '.jpg', dpi=400, bbox_inches='tight')
    plt.close()

In [33]:
# 集成 k-means 聚类
def my_k_cluster(x, k):
    klf = cluster.KMeans(n_clusters=k, random_state=10)
    klf.fit(x)
    y_pre = klf.predict(x)

    return y_pre, klf

In [34]:
def pplot3d(klf, x, index, label, huahe, name):
    fig = plt.figure(dpi=400)
    ax = plt.axes(projection='3d')

    b = klf.cluster_centers_
    print(name + '中心点坐标:{}'.format(b))
    ax.scatter3D(x[index, huahe[0]], x[index, huahe[1]], x[index, huahe[2]], color='r', marker='o')
    ax.scatter3D(x[index==False, huahe[0]], x[index==False, huahe[1]], x[index==False, huahe[2]], color='b', marker='*')
    
    ax.scatter3D(b[0, huahe[0]], b[0, huahe[1]], b[0, huahe[2]], color='r', marker='+')
    # ax.text(b[0, huahe[0]], b[0, huahe[1]], b[0, huahe[2]], (b[0, huahe[0]], b[0, huahe[1]], b[0, huahe[2]]))
    ax.scatter3D(b[1, huahe[0]], b[1, huahe[1]], b[1, huahe[2]], color='b', marker='+')
    # ax.text(b[1, huahe[0]], b[1, huahe[1]], b[1, huahe[2]], (b[1, huahe[0]], b[1, huahe[1]], b[1, huahe[2]]))
    ax.set(xlabel=label[0], ylabel=label[1], zlabel=label[2])

    plt.savefig('./imgs/问题二/'+ name +'聚类空间.jpg', dpi=400, bbox_inches='tight')
    plt.close()

### 高钾 无风化

In [35]:
data221 = pd.read_excel('./data/gaojia_wufenghua.xlsx')
data221.head()

,文物编号,纹饰,类型,颜色,表面风化,文物采样点,采样类别,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),...,氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2),二氧化硫(SO2),成分总和
0,1,C,高钾,蓝绿,无风化,01,无风化,69.33,0.0,9.99,...,3.93,1.74,3.87,0.00,0.00,1.17,0.00,0.0,0.39,97.61
1,3,A,高钾,蓝绿,无风化,03部位1,无风化,87.05,0.0,5.19,...,4.06,0.00,0.78,0.25,0.00,0.66,0.00,0.0,0.00,100.00
2,3,A,高钾,蓝绿,无风化,03部位2,无风化,61.71,0.0,12.37,...,5.50,2.16,5.09,1.41,2.86,0.70,0.10,0.0,0.00,98.88
3,4,A,高钾,蓝绿,无风化,04,无风化,65.88,0.0,9.67,...,6.44,2.06,2.18,0.00,0.00,0.79,0.00,0.0,0.36,96.06
4,5,A,高钾,蓝绿,无风化,05,无风化,61.58,0.0,10.95,...,7.50,2.62,3.27,0.00,0.00,0.94,0.06,0.0,0.47,96.51


In [36]:
df221 = data221.iloc[:, 7:-1]
# 方差
df221 = df221[df221.std(axis=0, ddof=1).sort_values(ascending=False)[:8].index]
name221 = df221.columns
elbow_analy(df221.values, '高钾-无风化')
y221, klf221 = my_k_cluster(df221, 2)
x221 = df221.values
[*zip(name221, [stats.levene(x221[y221==0, i], x221[y221==1, i])[0:2] for i in range(x221.shape[1])])]

[('二氧化硅(SiO2)', (0.10950519668524637, 0.747537622282407)),
 ('氧化钾(K2O)', (1.2478925530192413, 0.29007570852942927)),
 ('氧化钙(CaO)', (0.004666330964728512, 0.9468850232064701)),
 ('氧化铝(Al2O3)', (0.5107185739147639, 0.49117391212831896)),
 ('氧化铁(Fe2O3)', (0.07596145498741215, 0.7884581791848906)),
 ('氧化铜(CuO)', (0.04075048350815549, 0.844067732463703)),
 ('五氧化二磷(P2O5)', (1.0032985877655605, 0.34013451418494944)),
 ('氧化钠(Na2O)', (1.1863855184948668, 0.30161171297587436))]

In [37]:
index221 = [1, 6, 7]
pplot3d(klf221, x221, y221==0, name221[index221], index221, '高钾-无风化')

高钾-无风化中心点坐标:[[63.62444444 10.81777778  6.36333333  7.34888889  2.31222222  2.81888889
   1.52333333  0.92666667]
 [81.06333333  4.87        2.24        4.43333333  0.79        1.35333333
   1.04        0.        ]]


### 高钾 风化

In [38]:
data222 = pd.read_excel('./data/gaojia_fenghua.xlsx')

In [39]:
df222 = data222.iloc[:, 7:-1]
df222 = df222[df222.std(axis=0, ddof=1).sort_values(ascending=False)[:8].index]
name222 = df222.columns
# elbow_analy(df222.values, '高钾-风化')
y222, klf222 = my_k_cluster(df222, 2)
x222 = df222.values
[*zip(name222, [stats.levene(x222[y222==0, i], x222[y222==1, i])[0:2] for i in range(x222.shape[1])])]

[('二氧化硅(SiO2)', (1.87125924680562, 0.2431421661490719)),
 ('氧化铝(Al2O3)', (0.801291551979674, 0.42130523170409284)),
 ('氧化铜(CuO)', (1.3437761386966776, 0.3108551616065252)),
 ('氧化钙(CaO)', (0.10366713681241181, 0.7635968800987182)),
 ('氧化钾(K2O)', (0.1612852417703575, 0.7085074894207763)),
 ('氧化镁(MgO)', (1.6542810985460423, 0.2677776715147498)),
 ('五氧化二磷(P2O5)', (0.03125000000000004, 0.8682736240923679)),
 ('氧化铁(Fe2O3)', (0.7272727272727273, 0.4418233076836298))]

In [40]:
index222 = [0, 2, 5]
pplot3d(klf222, x222, y222==0, name222[index222], index222, '高钾-风化')

高钾-风化中心点坐标:[[95.36        1.19666667  1.34666667  0.51666667  0.84        0.
   0.16666667  0.29      ]
 [92.56666667  2.66333333  1.77666667  1.22333333  0.24666667  0.39333333
   0.39333333  0.24      ]]


### 铅钡 无风化

In [41]:
data223 = pd.read_excel('./data/qianbei_wufenghua.xlsx')
data223.head()

,文物编号,纹饰,类型,颜色,表面风化,文物采样点,采样类别,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),...,氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2),二氧化硫(SO2),成分总和
0,20,A,铅钡,浅蓝,无风化,20,无风化,37.36,0.00,0.71,...,5.45,1.51,4.78,9.30,23.55,5.75,0.00,0.00,0.0,88.41
1,23,A,铅钡,蓝绿,风化,23未风化点,无风化,53.79,7.92,0.00,...,1.42,0.00,2.99,16.98,11.86,0.00,0.33,0.00,0.0,96.50
2,24,C,铅钡,紫,无风化,24,无风化,31.94,0.00,0.00,...,1.59,0.00,8.46,29.14,26.23,0.14,0.91,0.00,0.0,98.88
3,25,C,铅钡,浅蓝,风化,25未风化点,无风化,50.61,2.31,0.00,...,1.90,1.55,1.12,31.90,6.65,0.19,0.20,0.00,0.0,97.06
4,28,A,铅钡,浅蓝,风化,28未风化点,无风化,68.08,0.00,0.26,...,4.70,0.41,0.33,17.14,4.04,1.04,0.12,0.23,0.0,98.69


In [42]:
df223 = data223.iloc[:, 7:-1]
df223 = df223[df223.std(axis=0, ddof=1).sort_values(ascending=False)[:8].index]
name223 = df223.columns
elbow_analy(df223.values, '铅钡-无风化')
y223, klf223 = my_k_cluster(df223, 2)
x223 = df223.values
[*zip(name223, [stats.levene(x223[y223==0, i], x223[y223==1, i])[0:2] for i in range(x223.shape[1])])]

[('二氧化硅(SiO2)', (1.101931837299736, 0.3057736635054501)),
 ('氧化铅(PbO)', (5.608142665018007, 0.027549899666424144)),
 ('氧化钡(BaO)', (0.45253459953845104, 0.5084691432831915)),
 ('氧化铝(Al2O3)', (3.369782046542806, 0.08060462755169646)),
 ('氧化钠(Na2O)', (0.20045516450231016, 0.6589381770329268)),
 ('氧化铜(CuO)', (2.1984979391870008, 0.15299797704234294)),
 ('五氧化二磷(P2O5)', (0.451937728439138, 0.5087459354274422)),
 ('氧化钙(CaO)', (1.5479440783667753, 0.22714769345944186))]

In [43]:
index223 = [1, 3, 5]
pplot3d(klf223, x223, y223==0, name223[index223], index223, '铅钡-无风化')

铅钡-无风化中心点坐标:[[44.95909091 27.58909091 11.98909091  3.61818182  1.91818182  2.06636364
   1.32363636  1.43090909]
 [63.55166667 17.03916667  6.26333333  5.22416667  1.46666667  0.85
   0.7975      1.21916667]]


### 铅钡 风化

In [44]:
# min - max 处理
data224 = pd.read_excel('./data/qianbei_fenghua.xlsx')
data224.head()

,文物编号,纹饰,类型,颜色,表面风化,文物采样点,采样类别,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),...,氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2),二氧化硫(SO2),成分总和
0,2,A,铅钡,浅蓝,风化,02,风化,36.28,0.0,1.05,...,5.73,1.86,0.26,47.43,0.00,3.57,0.19,0.0,0.00,99.89
1,8,C,铅钡,紫,风化,08,风化,20.14,0.0,0.00,...,1.34,0.00,10.41,28.68,31.23,3.59,0.37,0.0,2.58,99.82
2,8,C,铅钡,紫,风化,08严重风化点,严重风化,4.61,0.0,0.00,...,1.11,0.00,3.14,32.45,30.62,7.56,0.53,0.0,15.03,98.24
3,11,C,铅钡,浅蓝,风化,11,风化,33.59,0.0,0.21,...,2.69,0.00,4.93,25.39,14.61,9.38,0.37,0.0,0.00,95.39
4,19,A,铅钡,NaN,风化,19,风化,29.64,0.0,0.00,...,3.57,1.33,3.51,42.82,5.35,8.83,0.19,0.0,0.00,98.76


In [45]:
df224 = data224.iloc[:, 7:-1]
df224 = df224[df224.std(axis=0, ddof=1).sort_values(ascending=False)[:8].index]
name224 = df224.columns
# elbow_analy(df224.values, '铅钡-风化')
y224, klf224 = my_k_cluster(df224, 2)
x224 = df224.values
[*zip(name224, [stats.levene(x224[y224==0, i], x224[y224==1, i])[0:2] for i in range(x224.shape[1])])]

[('氧化铅(PbO)', (2.7596849596178736, 0.10967672596426198)),
 ('二氧化硅(SiO2)', (0.10460486690656694, 0.7491734226408205)),
 ('氧化钡(BaO)', (1.5376252851470735, 0.22695910647775686)),
 ('二氧化硫(SO2)', (5563.791961604785, 6.365413942000836e-30)),
 ('五氧化二磷(P2O5)', (4.285303856207117, 0.04936856081880652)),
 ('氧化铜(CuO)', (16.417481654907416, 0.0004621664635659379)),
 ('氧化铝(Al2O3)', (1.5685807091985056, 0.22248058782289962)),
 ('氧化钙(CaO)', (1.4573189074281212, 0.23912074642195938))]

In [46]:
index224 = [1, 2, 4]
pplot3d(klf224, x224, y224==0, name224[index224], index224, '铅钡-风化')

铅钡-风化中心点坐标:[[45.70818182 27.24863636  8.06545455  0.          5.31318182  1.42954545
   3.31318182  2.77090909]
 [30.145      12.065      32.3875      8.88        5.08        6.93
   1.0825      2.28      ]]


## 问3：对分类结果的合理性和敏感性进行分析

### 高钾 无风化

In [47]:
num221 = df221.shape[0]
index221 = np.random.choice(num221, np.ceil(num221*0.1).astype(dtype=np.int32), replace=False)
df221_1 = df221.drop(index=index221).reset_index(drop=True)
y221_1, klf221_1 = my_k_cluster(df221_1, 2)
x221_1 = df221_1.values
[*zip(name221, [stats.levene(x221_1[y221_1==0, i], x221_1[y221_1==1, i])[0:2] for i in range(x221_1.shape[1])])]

[('二氧化硅(SiO2)', (1.5493840680944215, 0.2484533533654268)),
 ('氧化钾(K2O)', (1.9703256742574837, 0.19801934342304706)),
 ('氧化钙(CaO)', (0.5504621613833561, 0.47934205271811464)),
 ('氧化铝(Al2O3)', (1.0345636852718278, 0.3388601179194778)),
 ('氧化铁(Fe2O3)', (0.6089420125989341, 0.45765143171001843)),
 ('氧化铜(CuO)', (1.5819053561671734, 0.2439565715084801)),
 ('五氧化二磷(P2O5)', (0.5335568363514669, 0.4859535199562733)),
 ('氧化钠(Na2O)', (0.3796433659183574, 0.5549165014783072))]

In [48]:
flag, counts = np.unique(y221_1 ^ np.delete(y221, index221), return_counts=True)
acc = counts / len(y221_1)
acc

array([1.])

### 高钾 风化

In [49]:
num222 = df222.shape[0]
index222 = np.random.choice(num222, np.ceil(num222*0.1).astype(dtype=np.int32), replace=False)
df222_1 = df222.drop(index=index222).reset_index(drop=True)
y222_1, klf222_1 = my_k_cluster(df222_1, 2)
x222_1 = df222_1.values
[*zip(name222, [stats.levene(x222_1[y222_1==0, i], x222_1[y222_1==1, i])[0:2] for i in range(x222_1.shape[1])])]

[('二氧化硅(SiO2)', (109.84502446981888, 0.0018545857306016419)),
 ('氧化铝(Al2O3)', (0.24204128751867457, 0.6564856870824572)),
 ('氧化铜(CuO)', (0.5968519957935169, 0.4960649491369566)),
 ('氧化钙(CaO)', (0.00421436004162332, 0.9523230016690475)),
 ('氧化钾(K2O)', (0.40105003652300963, 0.5715287456900479)),
 ('氧化镁(MgO)', (0.9925686591276252, 0.3925443682180632)),
 ('五氧化二磷(P2O5)', (0.3868421052631578, 0.5780298871184445)),
 ('氧化铁(Fe2O3)', (0.5785714285714291, 0.502193476219123))]

In [50]:
flag, counts = np.unique(y222_1 ^ np.delete(y222, index222), return_counts=True)
acc = counts / len(y222_1)
acc

array([1.])

### 铅钡 无风化

In [51]:
num223 = df223.shape[0]
index223 = np.random.choice(num223, np.ceil(num223*0.1).astype(dtype=np.int32), replace=False)
df223_1 = df223.drop(index=index223).reset_index(drop=True)
y223_1, klf223_1 = my_k_cluster(df223_1, 2)
x223_1 = df223_1.values
[*zip(name223, [stats.levene(x223_1[y223_1==0, i], x223_1[y223_1==1, i])[0:2] for i in range(x223_1.shape[1])])]

[('二氧化硅(SiO2)', (0.014651702474807198, 0.9049970014409511)),
 ('氧化铅(PbO)', (0.0004637767013701356, 0.9830554392205771)),
 ('氧化钡(BaO)', (0.761406386805718, 0.3943801203638184)),
 ('氧化铝(Al2O3)', (1.0812660803153535, 0.31218133517123314)),
 ('氧化钠(Na2O)', (1.2417845229466389, 0.27978614463848583)),
 ('氧化铜(CuO)', (2.0280046073455322, 0.1715287915853594)),
 ('五氧化二磷(P2O5)', (1.3350520726003081, 0.263016343488924)),
 ('氧化钙(CaO)', (5.140890010592251, 0.03591936466384747))]

In [52]:
flag, counts = np.unique(y223_1 ^ np.delete(y223, index223), return_counts=True)
acc = counts / len(y223_1)
acc

array([0.2, 0.8])

### 铅钡 风化

In [53]:
num224 = df224.shape[0]
index224 = np.random.choice(num224, np.ceil(num224*0.1).astype(dtype=np.int32), replace=False)
df224_1 = df224.drop(index=index224).reset_index(drop=True)
y224_1, klf224_1 = my_k_cluster(df224_1, 2)
x224_1 = df224_1.values
[*zip(name224, [stats.levene(x224_1[y224_1==0, i], x224_1[y224_1==1, i])[0:2] for i in range(x224_1.shape[1])])]

[('氧化铅(PbO)', (0.3718714951374558, 0.5485282364348267)),
 ('二氧化硅(SiO2)', (3.031594598856528, 0.09629008207397309)),
 ('氧化钡(BaO)', (2.693661941821594, 0.11564102353173954)),
 ('二氧化硫(SO2)', (8.721864394763976, 0.007588856569836601)),
 ('五氧化二磷(P2O5)', (1.443649217865418, 0.24292552362469697)),
 ('氧化铜(CuO)', (12.479765394479005, 0.001974111812742673)),
 ('氧化铝(Al2O3)', (2.1521376765272575, 0.15718861498692344)),
 ('氧化钙(CaO)', (2.910039814474172, 0.10277795573083495))]

In [54]:
flag, counts = np.unique(y224_1 ^ np.delete(y224, index224), return_counts=True)
acc = counts / len(y224_1)
acc

array([0.91304348, 0.08695652])

# 问题三

In [55]:
import pandas as pd
import numpy as np
import copy
import matplotlib.pyplot as plt
from sklearn import preprocessing, cluster

plt.rcParams['font.sans-serif'] = ['SimHei']  # 中文字体设置-黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决保存图像是负号'-'显示为方块的问题
# sns.set(font='SimHei', font_scale=0.8)        # 解决Seaborn中文显示问题
data31 = pd.read_excel("./data/predict.xlsx")
data31.fillna(0, inplace=True)
data31

,文物编号,表面风化,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),氧化钙(CaO),氧化镁(MgO),氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2),二氧化硫(SO2)
0,A1,无风化,78.45,0.0,0.00,6.08,1.86,7.23,2.15,2.11,0.00,0.00,1.06,0.03,0.00,0.51
1,A2,风化,37.75,0.0,0.00,7.63,0.00,2.33,0.00,0.00,34.30,0.00,14.27,0.00,0.00,0.00
2,A3,无风化,31.95,0.0,1.36,7.19,0.81,2.93,7.06,0.21,39.58,4.69,2.68,0.52,0.00,0.00
3,A4,无风化,35.47,0.0,0.79,2.89,1.05,7.07,6.45,0.96,24.28,8.31,8.45,0.28,0.00,0.00
4,A5,风化,64.29,1.2,0.37,1.64,2.34,12.75,0.81,0.94,12.23,2.16,0.19,0.21,0.49,0.00
5,A6,风化,93.17,0.0,1.35,0.64,0.21,1.52,0.27,1.73,0.00,0.00,0.21,0.00,0.00,0.00
6,A7,风化,90.83,0.0,0.98,1.12,0.00,5.06,0.24,1.17,0.00,0.00,0.13,0.00,0.00,0.11
7,A8,无风化,51.12,0.0,0.23,0.89,0.00,2.12,0.00,9.01,21.24,11.34,1.46,0.31,0.00,2.26


## 问1：对附件表单 3 中未知类别玻璃文物的化学成分进行分析，鉴别其所属类型

### 风化 待预测

In [56]:
fenghua311 = data31.loc[data31['表面风化'] == '风化', :]
fenghua311

,文物编号,表面风化,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),氧化钙(CaO),氧化镁(MgO),氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2),二氧化硫(SO2)
1,A2,风化,37.75,0.0,0.00,7.63,0.00,2.33,0.00,0.00,34.30,0.00,14.27,0.00,0.00,0.00
4,A5,风化,64.29,1.2,0.37,1.64,2.34,12.75,0.81,0.94,12.23,2.16,0.19,0.21,0.49,0.00
5,A6,风化,93.17,0.0,1.35,0.64,0.21,1.52,0.27,1.73,0.00,0.00,0.21,0.00,0.00,0.00
6,A7,风化,90.83,0.0,0.98,1.12,0.00,5.06,0.24,1.17,0.00,0.00,0.13,0.00,0.00,0.11


In [57]:
# 风化 - 高钾 or 铅钡 解法 211
# {0:'高钾', 1:'铅钡'}
leibie311 = clf211.predict(fenghua311.iloc[:, 2:])
leibie311_pre = []
for item in leibie311:
    if item==0:
        leibie311_pre.append('高钾') 
    else:
        leibie311_pre.append('铅钡')
leibie311_pre = np.array(leibie311_pre)
yalei311 = copy.deepcopy(leibie311_pre)
leibie311_pre

array(['铅钡', '铅钡', '高钾', '高钾'], dtype='<U2')

In [58]:
# 风化 - 高钾 - 聚类 解法 222
yalei311[leibie311_pre=='高钾']  = klf222.predict(fenghua311.loc[leibie311_pre=='高钾', name222])
# 风化 - 铅钡 - 聚类 解法 224
yalei311[leibie311_pre=='铅钡']  = klf224.predict(fenghua311.loc[leibie311_pre=='铅钡', name224])

In [59]:
df311 = pd.concat([pd.DataFrame(np.c_[leibie311_pre, yalei311], columns=['类别', '亚类']), fenghua311.reset_index(drop=True)], axis=1)
df311.head()

,类别,亚类,文物编号,表面风化,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),氧化钙(CaO),氧化镁(MgO),氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2),二氧化硫(SO2)
0,铅钡,0,A2,风化,37.75,0.0,0.00,7.63,0.00,2.33,0.00,0.00,34.30,0.00,14.27,0.00,0.00,0.00
1,铅钡,0,A5,风化,64.29,1.2,0.37,1.64,2.34,12.75,0.81,0.94,12.23,2.16,0.19,0.21,0.49,0.00
2,高钾,1,A6,风化,93.17,0.0,1.35,0.64,0.21,1.52,0.27,1.73,0.00,0.00,0.21,0.00,0.00,0.00
3,高钾,1,A7,风化,90.83,0.0,0.98,1.12,0.00,5.06,0.24,1.17,0.00,0.00,0.13,0.00,0.00,0.11


### 无风化 待预测

In [60]:
wufeng312 = data31.loc[data31['表面风化'] == '无风化', :]
wufeng312

,文物编号,表面风化,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),氧化钙(CaO),氧化镁(MgO),氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2),二氧化硫(SO2)
0,A1,无风化,78.45,0.0,0.00,6.08,1.86,7.23,2.15,2.11,0.00,0.00,1.06,0.03,0.0,0.51
2,A3,无风化,31.95,0.0,1.36,7.19,0.81,2.93,7.06,0.21,39.58,4.69,2.68,0.52,0.0,0.00
3,A4,无风化,35.47,0.0,0.79,2.89,1.05,7.07,6.45,0.96,24.28,8.31,8.45,0.28,0.0,0.00
7,A8,无风化,51.12,0.0,0.23,0.89,0.00,2.12,0.00,9.01,21.24,11.34,1.46,0.31,0.0,2.26


In [61]:
# 无风化 - 高钾 or 铅钡 解法 212
# {0:'高钾', 1:'铅钡'}
leibie312 = clf212.predict(wufeng312.iloc[:, 2:])
leibie312_pre = []
for item in leibie312:
    if item==0:
        leibie312_pre.append('高钾') 
    else:
        leibie312_pre.append('铅钡')
leibie312_pre = np.array(leibie312_pre)
yalei312 = copy.deepcopy(leibie312_pre)
leibie312_pre

array(['高钾', '铅钡', '铅钡', '铅钡'], dtype='<U2')

In [62]:
# 无风化 - 高钾 - 聚类 解法 221
yalei312[leibie312_pre=='高钾'] = klf221.predict(wufeng312.loc[leibie312_pre=='高钾', name221])
# 无风化 - 铅钡 - 聚类 解法 223
yalei312[leibie312_pre=='铅钡'] = klf223.predict(wufeng312.loc[leibie312_pre=='铅钡', name223])

In [63]:
df312 = pd.concat([pd.DataFrame(np.c_[leibie312_pre, yalei312], columns=['类别', '亚类']), wufeng312.reset_index(drop=True), ], axis=1)
df312.head()

,类别,亚类,文物编号,表面风化,二氧化硅(SiO2),氧化钠(Na2O),氧化钾(K2O),氧化钙(CaO),氧化镁(MgO),氧化铝(Al2O3),氧化铁(Fe2O3),氧化铜(CuO),氧化铅(PbO),氧化钡(BaO),五氧化二磷(P2O5),氧化锶(SrO),氧化锡(SnO2),二氧化硫(SO2)
0,高钾,1,A1,无风化,78.45,0.0,0.00,6.08,1.86,7.23,2.15,2.11,0.00,0.00,1.06,0.03,0.0,0.51
1,铅钡,0,A3,无风化,31.95,0.0,1.36,7.19,0.81,2.93,7.06,0.21,39.58,4.69,2.68,0.52,0.0,0.00
2,铅钡,0,A4,无风化,35.47,0.0,0.79,2.89,1.05,7.07,6.45,0.96,24.28,8.31,8.45,0.28,0.0,0.00
3,铅钡,0,A8,无风化,51.12,0.0,0.23,0.89,0.00,2.12,0.00,9.01,21.24,11.34,1.46,0.31,0.0,2.26


## 问2：对分类结果的敏感性进行分析

In [64]:
df32 = pd.concat([df311, df312]).reset_index(drop=True)
target32 = df32.iloc[:, 0:4]
huahe32 = df32.iloc[:, 4:]

In [65]:
temp32_1 = huahe32
temp32_2 = target32
for _ in range(1000):
    temp = huahe32 + huahe32.std().values * np.random.uniform(low=-1, high=1, size=huahe32.shape) * 1.0
    temp32_1 = pd.concat([temp32_1, temp]).reset_index(drop=True)
    temp32_2 = pd.concat([temp32_2, target32]).reset_index(drop=True)

In [66]:
all_data32 = pd.concat([temp32_2.replace({'高钾':0, '铅钡':1}), temp32_1.clip(lower=0, upper=105)], axis=1).reset_index(drop=True)
all_data32 =  all_data32.loc[all_data32.iloc[:, 4:].sum(axis=1) <= 105, :]
all_data32 = all_data32.loc[all_data32.iloc[:, 4:].sum(axis=1) >= 85, :].reset_index(drop=True)

In [67]:
# 无风化
wufeng32 = all_data32.loc[all_data32['表面风化'] == '无风化', :]
# {0:'高钾', 1:'铅钡'}
leibie32_pre = clf212.predict(wufeng32.iloc[:, 4:])
yalei32_pre = copy.deepcopy(leibie32_pre)
# 无风化 - 高钾 - 聚类 解法 221
yalei32_pre[leibie32_pre==0] = klf222.predict(wufeng32.loc[leibie32_pre==0, name222])
# 无风化 - 铅钡 - 聚类 解法 223
yalei32_pre[leibie32_pre==1] = klf224.predict(wufeng32.loc[leibie32_pre==1, name224])
leibie32 = wufeng32['类别']
yalei32 = wufeng32['亚类']

In [68]:
acc = []
for i, j in zip(leibie32_pre - leibie32.values, (yalei32_pre - yalei32.astype(dtype=np.int32)).values):
    if i == 0:
        if j == 0:
            acc.append(1)
        else:
            acc.append(0)
    else:
        acc.append(0)
score1 = sum(acc)/len(acc)
score1

0.9575596816976127

In [69]:
# 风化
wufeng32 = all_data32.loc[all_data32['表面风化'] == '风化', :]
# {0:'高钾', 1:'铅钡'}
leibie32_pre = clf211.predict(wufeng32.iloc[:, 4:])
yalei32_pre = copy.deepcopy(leibie32_pre)
# 风化 - 高钾 - 聚类 解法 222
yalei32_pre[leibie32_pre==0] = klf221.predict(wufeng32.loc[leibie32_pre==0, name221])
# 风化 - 铅钡 - 聚类 解法 224
yalei32_pre[leibie32_pre==1] = klf223.predict(wufeng32.loc[leibie32_pre==1, name223])
leibie32 = wufeng32['类别']
yalei32 = wufeng32['亚类']
acc = []
for i, j in zip(leibie32_pre - leibie32.values, (yalei32_pre - yalei32.astype(dtype=np.int32)).values):
    if i == 0:
        if j == 0:
            acc.append(1)
        else:
            acc.append(0)
    else:
        acc.append(0)
score2 = sum(acc)/len(acc)
score2

0.7565392354124748

# 问题四

## 问1：针对不同类别的玻璃文物样品，分析其化学成分之间的关联关系

In [70]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

plt.rcParams['font.sans-serif'] = ['SimHei']  # 中文字体设置-黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决保存图像是负号'-'显示为方块的问题
sns.set(font='SimHei')  # 解决Seaborn中文显示问题

In [71]:
def pplot_heatmap(x, names, leixing, methods):
    for item in methods:
        xiangguan = x.corr(method=item)
        plt.figure(dpi=400)
        H = sns.heatmap(data=xiangguan,
                        xticklabels=names, yticklabels=names,
                        cmap=plt.get_cmap('Reds'),
                        annot=True, fmt=".2f",
                        annot_kws={'size':8, 'weight':'normal'}) 
        plt.savefig('./imgs/问题四/' + leixing + item + '-相关系数.jpg', dpi=400, bbox_inches='tight')
        plt.close()  
        
methods = ['pearson', 'kendall', 'spearman']

### 高钾 相关性分析

#### 高钾 - 风化数据

In [72]:
# 风化数据
data4111 = pd.read_excel('./data/gaojia_fenghua.xlsx')
data4111.head()
x4111 = data4111.iloc[:, 7:-1]
# Pearson 相关系数
# Kendall tau 相关系数
# Kendall tau 相关系数
pplot_heatmap(x=x4111, names=x4111.columns, leixing='高钾-风化', methods=methods)

#### 高钾 - 无风化数据

In [73]:
data4112 = pd.read_excel('./data/gaojia_wufenghua.xlsx')
data4112.head()
x4112 = data4112.iloc[:, 7:-1]
# Pearson 相关系数
# Kendall tau 相关系数
# Kendall tau 相关系数
pplot_heatmap(x=x4112, names=x4112.columns, leixing='高钾-无风化', methods=methods)

### 铅钡 相关性分析

#### 铅钡 - 风化数据

In [74]:
# 风化数据
data4121 = pd.read_excel('./data/qianbei_fenghua.xlsx')
data4121.head()
x4121 = data4121.iloc[:, 7:-1]
# Pearson 相关系数
# Kendall tau 相关系数
# Kendall tau 相关系数
pplot_heatmap(x=x4121, names=x4121.columns, leixing='铅钡-风化', methods=methods)

#### 铅钡 - 无风化数据

In [75]:
data4122 = pd.read_excel('./data/qianbei_wufenghua.xlsx')
data4122.head()
x4122 = data4122.iloc[:, 7:-1]
# Pearson 相关系数
# Kendall tau 相关系数
# Kendall tau 相关系数
pplot_heatmap(x=x4122, names=x4122.columns, leixing='铅钡-无风化', methods=methods)

## 问2：比较不同类别之间的化学成分关联关系的差异性

In [76]:
#文字分析